In [1]:
import pyspark
import numpy as np
from datetime import datetime as dt
from datetime import time

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
# vuelos_programados = id_vuelo, id_aerolinea, id_aeropuerto_salida, id_aeropuerto_llegada, fecha_salida, hora_salida, fecha_llegada, hora_llegada
# estado_vuelos = id_vuelo, id_aerolinea, fecha_salida, hora_salida, fecha_llegada, hora_llegada, estado

# a) Hallar el aeropuerto con mayor transito
# b) Hallar la aerolinea con mayor cantidad de vuelos
# c) Hallar la aerolinea con mayor cantidad de cancelaciones
# d) Hallar el vuelo con mayor retraso en la hora de salida
# e) Hallar el vuelo con mayor retraso en la hora de llegada
# f) Hallar la aerolinea mas puntual
# g) Hallar el aeropuerto con mayor desviacion respecto del horario esperado de llegada

In [3]:
vuelos_programados = [
    ('EQ101B', 1, 1, 2, dt(2015, 5, 12), time(23,45,0), dt(2015, 5, 13), time(3, 0, 0)),
    ('BR205A', 2, 2, 1, dt(2017, 3, 22), time(3,0,0), dt(2017, 3, 22), time(23, 30, 0)),
    ('EQ312B', 1, 2, 1, dt(2013, 5, 12), time(11,45,0), dt(2015, 5, 12), time(22, 0, 0)),
    ('BR112A', 2, 1, 2, dt(2019, 11, 2), time(11,0,0), dt(2017, 3, 22), time(17, 0, 0)),
    ('BR113A', 2, 1, 3, dt(2019, 11, 2), time(11,0,0), dt(2017, 3, 22), time(17, 0, 0))
]

estado_vuelos = [
    ('EQ101B', 1, dt(2015, 5, 12), time(23,45,0), dt(2015, 5, 13), time(3, 0, 0), 'a tiempo'),
    ('BR205A', 2, dt(2017, 3, 22), time(3,0,0), dt(2017, 3, 23), time(23, 30, 0), 'a tiempo'),
    ('EQ312B', 1, dt(2013, 5, 12), time(13,0,0), dt(2015, 5, 12), time(23, 15, 0), 'retrasado'),
    ('BR112A', 2, dt(2019, 11, 2), time(11,0,0), dt(2017, 3, 22), time(17, 0, 0), 'a tiempo'),
    ('BR113A', 2, None, None, None, None, 'cancelado')
]

programados_rdd = sc.parallelize(vuelos_programados)
estados_rdd = sc.parallelize(estado_vuelos)

In [4]:
aeropuerto_mayor_transito = programados_rdd.flatMap(lambda x: ((x[2], 1), (x[3], 1)))\
                                           .reduceByKey(lambda x,y: x+y)\
                                           .top(1, key=lambda x: x[1])

print("El aeropuerto de mayor transito es el #%i con %i vuelos" % aeropuerto_mayor_transito[0])

El aeropuerto de mayor transito es el #1 con 5 vuelos


In [5]:
aerolinea_con_mas_vuelos = programados_rdd.map(lambda x: (x[1], 1))\
                                          .reduceByKey(lambda x,y: x+y)\
                                          .top(1, key=lambda x: x[0])

print("La aerolinea con mas vuelos es la #%i con %i vuelos" % aerolinea_con_mas_vuelos[0])

La aerolinea con mas vuelos es la #2 con 3 vuelos


In [7]:
aerolinea_con_mas_cancelados = estados_rdd.filter(lambda x: x[6] == 'cancelado')\
                                          .map(lambda x: (x[1], 1) )\
                                          .reduceByKey(lambda x,y: x+y)\
                                          .top(1, key= lambda x: x[1])

print("La aerolinea con mas vuelos cancelados es la #%i con %i vuelos" % aerolinea_con_mas_cancelados[0])

La aerolinea con mas vuelos cancelados es la #2 con 1 vuelos


In [18]:
salidas_programadas = programados_rdd.map(lambda x: (x[0], x[5]))

vuelo_con_mayor_retraso_salida = estados_rdd.map(lambda x: (x[0], x[3]))\
                                            .join(salidas_programadas)\
                                            .map(lambda x: (x[0], x[1]) )\
                                            .map(lambda x: (x[0], x[1][0] - x[1][1]) )\
                                            .top(1, key=lambda x: x[1])

print("El vuelo con mayor retraso es el %s" % vuelo_con_mayor_retraso_salida[0][0])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 43.0 failed 1 times, most recent failure: Lost task 1.0 in stage 43.0 (TID 142, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/fedoremelianenko/.spark-2.4.2-bin-hadoop2.7/python/pyspark/rdd.py", line 839, in func
    initial = next(ite